# Creating a Sampled Dataset

**Learning Objectives**

1. Setup up the environment
1. Sample the natality dataset to create train, eval, test sets
1. Preprocess the data in Pandas dataframe


## Introduction 
In this notebook, we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe for a small, repeatable sample.

We will set up the environment, sample the natality dataset to create train, eval, test splits, and preprocess the data in a Pandas dataframe.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/tree/master/courses/machine_learning/deepdive2/end_to_end_ml/solutions/sample_babyweight.ipynb).

## Set up environment variables and load necessary libraries

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
!pip install --user google-cloud-bigquery==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 KB 20.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 2.0.0 requires google-resumable-media>=1.3.0, but you have google-resumable-media 0.5.1 which is incompatible.
google-cloud-aiplatform 1.9.0 requires google-cloud-storage<2.0.0dev,>=1.32.0, but you have google-cloud-storage 2.0.0 which is incompatible.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.36.0 which is incompatible.


**Note**: Restart your kernel to use updated packages.

Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

Import necessary libraries.

In [1]:
from google.cloud import bigquery
import pandas as pd

**Lab Task #1:** Set up environment variables so that we can use them throughout the notebook

In [3]:
%%bash
# TODO 1
export BUCKET='qwiklabs-gcp-02-0cdb8ea82ebd'
export PROJECT='qwiklabs-gcp-02-0cdb8ea82ebd'
export REGION='australia-southeast1'

echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-02-0cdb8ea82ebd


In [4]:
PROJECT = 'qwiklabs-gcp-02-0cdb8ea82ebd'  # Replace with your PROJECT

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets. Let's create a BigQuery client that we'll use throughout the lab.

In [5]:
bq = bigquery.Client(project = PROJECT)

We need to figure out the right way to divide our hash values to get our desired splits. To do that we need to define some values to hash within the module. Feel free to play around with these values to get the perfect combination.

In [6]:
modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

train_buckets, eval_buckets

(80, 10)

We can make a series of queries to check if our bucketing values result in the correct sizes for each of our dataset splits and then adjust accordingly.  
Therefore, to make our code more compact and reusable, let's define a function to return the head of a dataframe produced from our queries up to a certain number of rows.

In [7]:
def display_dataframe_head_from_query(query, count=10):
    """Displays count rows from dataframe head from query.
    
    Args:
        query: str, query to be run on BigQuery, results stored in dataframe.
        count: int, number of results from head of dataframe to display.
    Returns:
        Dataframe head with count number of results.
    """
    df = bq.query(
        query + " LIMIT {limit}".format(limit=count)
    ).to_dataframe()
    return df.head(count)

For our first query, we're going to use the original query above to get our label, features, and columns to combine into our hash which we will use to perform our repeatable splitting. There are only a limited number of years, months, days, and states in the dataset. Let's see what the hash values are. We will need to include all of these extra columns to hash on to get a fairly uniform spread of the data. Feel free to try less or more in the hash and see how it changes your results.

In [8]:
# Get label, features, and columns to hash and split into buckets
hash_cols_fixed_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
        WHEN day IS NULL THEN
            CASE
                WHEN wday IS NULL THEN 0
                ELSE wday
            END
        ELSE day
    END AS date,
    IFNULL(state, "Unknown") AS state,
    IFNULL(mother_birth_state, "Unknown") AS mother_birth_state
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0
"""

display_dataframe_head_from_query(hash_cols_fixed_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,year,month,date,state,mother_birth_state
0,7.063611,True,32,1,37,2001,12,3,CO,CA
1,4.687028,True,30,3,33,2001,6,5,IN,IN
2,7.561856,True,20,1,39,2001,4,5,MN,MN
3,7.561856,True,31,1,37,2001,10,5,MS,MS
4,7.312733,True,32,1,40,2001,11,3,MO,MO
5,7.627994,False,30,1,40,2001,10,5,NY,PA
6,7.251004,True,33,1,37,2001,11,5,WA,WA
7,7.500126,False,23,1,39,2001,9,2,OK,LA
8,7.125340,False,33,1,39,2001,1,4,TX,MS
9,7.749249,True,31,1,39,2001,1,1,TX,Foreign


Using `COALESCE` would provide the same result as the nested `CASE WHEN`. This is preferable when all we want is the first non-null instance.  
To be precise the `CASE WHEN` would become `COALESCE(wday, day, 0) AS date`. You can read more about it [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/conditional_expressions).

Next query will combine our hash columns and will leave us just with our label, features, and our hash values.

In [9]:
data_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    ) AS hash_values
FROM
    ({CTE_hash_cols_fixed})
""".format(CTE_hash_cols_fixed=hash_cols_fixed_query)

display_dataframe_head_from_query(data_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,7.063611,True,32,1,37,4762325092919148672
1,4.687028,True,30,3,33,2341060194216507348
2,7.561856,True,20,1,39,-8842767231851202242
3,7.561856,True,31,1,37,7957807816914159435
4,7.312733,True,32,1,40,-5961624242430066305
5,7.627994,False,30,1,40,5493295634082918412
6,7.251004,True,33,1,37,-2988893757655690534
7,7.500126,False,23,1,39,-6735199252008114417
8,7.125340,False,33,1,39,-3514093303120687641
9,7.749249,True,31,1,39,2175328516857391398


The next query is going to find the counts of each of the unique 657484 `hash_values`. This will be our first step at making actual hash buckets for our split via the `GROUP BY`.

In [10]:
# Get the counts of each of the unique hash of our splitting column
first_bucketing_query = """
SELECT hash_values, COUNT(*) AS num_records
FROM ({CTE_data})
GROUP BY hash_values
""".format(CTE_data=data_query)

display_dataframe_head_from_query(first_bucketing_query)

,hash_values,num_records
0,4100584960946904090,74
1,-6873880094893634931,333
2,8813898149905115516,382
3,6800313576194062135,1738
4,4002725964095333770,130
5,-7916673508712224858,28
6,-4240189934893571269,437
7,7786256100450198866,1273
8,-3262319040723013338,514
9,-8219217410966427260,174


The query below performs a second layer of bucketing where now for each of these bucket indices we count the number of records.

In [11]:
# Get the number of records in each of the hash buckets
second_bucketing_query = """
SELECT ABS(MOD(hash_values, {modulo_divisor})) AS bucket_index,
    SUM(num_records) AS num_records
FROM ({CTE_first_bucketing})
GROUP BY ABS(MOD(hash_values, {modulo_divisor}))
""".format(CTE_first_bucketing=first_bucketing_query, modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(second_bucketing_query)

,bucket_index,num_records
0,23,559019
1,35,250505
2,21,247072
3,74,480999
4,54,256517
5,53,230298
6,36,246041
7,73,411771
8,71,260774
9,87,523881


The number of records is hard for us to easily understand the split, so we will normalize the count into percentage of the data in each of the hash buckets in the next query.

In [12]:
# Calculate the overall percentages
percentages_query = """
SELECT bucket_index, num_records,
    CAST(num_records AS FLOAT64) / (
      SELECT SUM(num_records)
      FROM ({CTE_second_bucketing})
    ) AS percent_records
FROM ({CTE_second_bucketing})
""".format(CTE_second_bucketing=second_bucketing_query)

display_dataframe_head_from_query(percentages_query)

,bucket_index,num_records,percent_records
0,52,204972,0.006209
1,33,410226,0.012427
2,79,403701,0.012229
3,65,289303,0.008764
4,75,367455,0.011131
5,47,393747,0.011928
6,59,361078,0.010938
7,83,411258,0.012458
8,41,244850,0.007417
9,0,277395,0.008403


We'll now select the range of buckets to be used in training.

In [13]:
# Choose hash buckets for training and pull in their statistics
train_query = """
SELECT *, "train" AS dataset_name
FROM ({CTE_percentages})
WHERE bucket_index >= 0
AND bucket_index < {train_buckets}
""".format(CTE_percentages=percentages_query, train_buckets=train_buckets)

display_dataframe_head_from_query(train_query)

,bucket_index,num_records,percent_records,dataset_name
0,4,398118,0.012060,train
1,17,222562,0.006742,train
2,64,283091,0.008576,train
3,39,224255,0.006793,train
4,43,201054,0.006090,train
5,48,370308,0.011218,train
6,45,265930,0.008056,train
7,62,426834,0.012930,train
8,38,338150,0.010243,train
9,68,197797,0.005992,train


We'll do the same by selecting the range of buckets to be used evaluation.

In [14]:
# Choose hash buckets for validation and pull in their statistics
eval_query = """
SELECT *, "eval" AS dataset_name
FROM ({CTE_percentages})
WHERE bucket_index >= {train_buckets}
AND bucket_index < {cum_eval_buckets}
""".format(CTE_percentages=percentages_query, train_buckets=train_buckets, cum_eval_buckets=train_buckets + eval_buckets)

display_dataframe_head_from_query(eval_query)

,bucket_index,num_records,percent_records,dataset_name
0,83,411258,0.012458,eval
1,89,256482,0.007770,eval
2,85,368045,0.011149,eval
3,87,523881,0.015870,eval
4,82,468179,0.014182,eval
5,84,341155,0.010334,eval
6,88,423809,0.012838,eval
7,81,233538,0.007074,eval
8,86,274489,0.008315,eval
9,80,312489,0.009466,eval


Lastly, we'll select the hash buckets to be used for the test split.

In [15]:
# Choose hash buckets for testing and pull in their statistics
test_query = """
SELECT *, "test" AS dataset_name
FROM ({CTE_percentages})
WHERE bucket_index >= {cum_eval_buckets}
AND bucket_index < {modulo_divisor}
""".format(CTE_percentages=percentages_query, cum_eval_buckets=train_buckets + eval_buckets, modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(test_query)

,bucket_index,num_records,percent_records,dataset_name
0,98,374697,0.011351,test
1,97,480790,0.014564,test
2,91,333267,0.010096,test
3,90,286465,0.008678,test
4,93,215710,0.006534,test
5,99,223334,0.006765,test
6,92,336735,0.010201,test
7,96,529357,0.016036,test
8,95,313544,0.009498,test
9,94,431001,0.013056,test


In the below query, we'll `UNION ALL` all of the datasets together so that all three sets of hash buckets will be within one table. We added `dataset_id` so that we can sort on it in the query after.

In [16]:
# Union the training, validation, and testing dataset statistics
union_query = """
SELECT 0 AS dataset_id, *
FROM ({CTE_train})
UNION ALL
SELECT 1 AS dataset_id, *
FROM ({CTE_eval})
UNION ALL
SELECT 2 AS dataset_id, *
FROM ({CTE_test})
""".format(CTE_train=train_query, CTE_eval=eval_query, CTE_test=test_query)

display_dataframe_head_from_query(union_query)

,dataset_id,bucket_index,num_records,percent_records,dataset_name
0,2,99,223334,0.006765,test
1,2,96,529357,0.016036,test
2,2,95,313544,0.009498,test
3,2,92,336735,0.010201,test
4,1,86,274489,0.008315,eval
5,1,81,233538,0.007074,eval
6,0,35,250505,0.007588,train
7,0,74,480999,0.014571,train
8,0,77,401941,0.012176,train
9,0,23,559019,0.016934,train


Lastly, we'll show the final split between train, eval, and test sets. We can see both the number of records and percent of the total data. It is really close to that we were hoping to get.

In [17]:
# Show final splitting and associated statistics
split_query = """
SELECT dataset_id, dataset_name, SUM(num_records) AS num_records, SUM(percent_records) AS percent_records
FROM ({CTE_union})
GROUP BY dataset_id, dataset_name
ORDER BY dataset_id
""".format(CTE_union=union_query)

display_dataframe_head_from_query(split_query)

,dataset_id,dataset_name,num_records,percent_records
0,0,train,25873134,0.783765
1,1,eval,3613325,0.109457
2,2,test,3524900,0.106778


Now that we know that our splitting values produce a good global splitting on our data, here's a way to get a well-distributed portion of the data in such a way that the train, eval, test sets do not overlap and takes a subsample of our global splits.

**Lab Task #2:** Sample the natality dataset

In [19]:
# TODO 2
# every_n allows us to subsample from each of the hash values
# This helps us get approximately the record counts we want

every_n = 1000
splitting_string = "ABS(MOD(hash_values, {0} * {1}))".format(every_n, modulo_divisor)

def create_data_split_sample_df(query_string, splitting_string, low, up):
    """Creates a dataframe with a sample of a data split.
    Args:
        query_string: str, query to run to generate splits.
        splitting_string: str, modulo string to split by.
        lo: float, lower bound for bucket filtering for split.
        up: float, upper bound for bucket filtering for split.
    Returns:
        Dataframe containing data split sample.
    """
    query = "SELECT * FROM ({0}) WHERE {1} >= {2} and {1} < {3}".format(
        query_string, splitting_string, int(low), int(up)
    )
    return bq.query(query).to_dataframe()

train_df = create_data_split_sample_df(data_query, splitting_string, low=0, up=train_percent)
eval_df = create_data_split_sample_df(data_query, splitting_string, low=train_percent, 
                                      up=train_percent + eval_percent)
test_df = create_data_split_sample_df(data_query, splitting_string, low=train_percent + eval_percent, 
                                      up=modulo_divisor)

print(f"There are {len(train_df)} examples in the train dataset.")
print(f"There are {len(eval_df)} examples in the validation dataset.")
print(f"There are {len(test_df)} examples in the test dataset.")

There are 7733 examples in the train dataset.
There are 1037 examples in the validation dataset.
There are 561 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`.  
Also, if there is more than one child we'll change the `plurality` to `Multiple(2+)`. While we're at it, we'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [20]:
train_df.head(7)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,7.625790,True,38,1,38,968753748112600022
1,7.363440,True,38,1,40,-6784884401981100070
2,7.561856,False,35,1,39,-6373125659687700000
3,5.249206,False,33,2,38,4896699230184800022
4,4.687028,True,15,1,36,5815559452861000023
5,3.688334,True,25,3,32,-6784884401981100070
6,7.672087,False,28,1,41,505732274561700014


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [21]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hash_values
count,7733.000000,7733.000000,7733.000000,7733.000000,7.733000e+03
mean,7.264415,28.213371,1.035691,38.691064,-2.984870e+17
std,1.303220,6.134232,0.201568,2.531921,5.590715e+18
min,0.562179,13.000000,1.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,1.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,1.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,1.000000,40.000000,4.896699e+18
max,11.563246,48.000000,4.000000,47.000000,9.203641e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

**Lab Task #3:** Preprocess the data in Pandas dataframe

In [22]:
# TODO 3

def preprocess(df):
    """ Preprocess pandas dataframe for augmented babyweight data.
    Args:
        df: Dataframe containing raw babyweight data.
    Returns:
        Pandas dataframe containing preprocessed raw babyweight data as well
            as simulated no ultrasound data masking some of the original data.
    """
    # Clean up raw data
    # Filter out what we don't want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)",
                    "Twins(2)",
                    "Triplets(3)",
                    "Quadruplets(4)",
                    "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace=True)

    no_ultrasound = df.copy(deep=True)    # Clone data and mask certain columns to simulate lack of ultrasound
    no_ultrasound["is_male"] = "Unknown"  # Modify is_male
    
    # Modify plurality
    condition = no_ultrasound["plurality"] != "Single(1)"
    no_ultrasound.loc[condition, "plurality"] = "Multiple(2+)"

    # Concatenate both datasets together and shuffle
    return pd.concat([df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

Let's process the train, eval, test set and see a small sample of the training data after our preprocessing:

In [23]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [24]:
train_df.head(7)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,7.782318,Unknown,22,Single(1),38,505732274561700014
1,8.875811,Unknown,26,Single(1),37,2054179216119800044
2,2.874828,Unknown,24,Single(1),31,4896699230184800022
3,5.202909,Unknown,25,Single(1),39,3804216552009300065
4,7.076839,False,28,Single(1),38,-4614303140002600076
5,7.500126,Unknown,24,Single(1),41,5815559452861000023
6,7.187070,False,32,Single(1),35,8482454469826500044


In [25]:
train_df.tail(7)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
15459,6.750554,Unknown,32,Single(1),38,7217843716311300005
15460,8.928722,Unknown,27,Single(1),35,-4614303140002600076
15461,8.187968,True,27,Single(1),39,-6848352519903300073
15462,7.438397,Unknown,28,Single(1),38,-1466514356607500060
15463,8.311427,Unknown,34,Single(1),38,-1796980678187600036
15464,7.874912,Unknown,31,Single(1),39,4896699230184800022
15465,7.874912,False,20,Single(1),39,-6820867331362700022


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [26]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hash_values
count,15466.000000,15466.000000,15466.000000,1.546600e+04
mean,7.264415,28.213371,38.691064,-2.984870e+17
std,1.303178,6.134034,2.531839,5.590534e+18
min,0.562179,13.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,40.000000,4.896699e+18
max,11.563246,48.000000,47.000000,9.203641e+18


## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

In [27]:
# Define columns
columns = ["weight_pounds",
           "is_male",
           "mother_age",
           "plurality",
           "gestation_weeks"]

# Write out CSV files
train_df.to_csv(path_or_buf="train.csv", columns=columns, header=False, index=False)
eval_df.to_csv(path_or_buf="eval.csv", columns=columns, header=False, index=False)
test_df.to_csv(path_or_buf="test.csv", columns=columns, header=False, index=False)

In [42]:
%%bash
wc -l *.csv

  2074 eval.csv
  1122 test.csv
 15466 train.csv
 18662 total


In [28]:
%%bash
head -7 *.csv

==> eval.csv <==
7.3744626639,False,25,Single(1),39
7.87491199864,True,30,Single(1),38
7.43839671988,True,36,Single(1),38
8.87581066812,True,20,Single(1),38
7.43839671988,False,20,Single(1),40
7.9586876582,Unknown,29,Single(1),38
6.93794738514,False,27,Single(1),39

==> test.csv <==
7.62578964258,Unknown,16,Single(1),39
6.8563763481999995,Unknown,20,Single(1),39
7.12534030784,Unknown,26,Single(1),39
7.5287862473,False,32,Single(1),37
5.81138522632,False,41,Single(1),36
6.9996768185,True,20,Single(1),39
7.62578964258,Unknown,26,Single(1),39

==> train.csv <==
7.7823178486,Unknown,22,Single(1),38
8.87581066812,Unknown,26,Single(1),37
2.87482789648,Unknown,24,Single(1),31
5.2029093832,Unknown,25,Single(1),39
7.0768386102,False,28,Single(1),38
7.50012615324,Unknown,24,Single(1),41
7.1870697412,False,32,Single(1),35


In [30]:
%%bash
tail -n7  *.csv

==> eval.csv <==
5.8753192823,True,24,Single(1),39
8.3114272774,False,31,Single(1),40
7.12534030784,True,17,Single(1),40
7.31273323054,Unknown,33,Single(1),40
8.62448368944,False,33,Single(1),40
7.50012615324,Unknown,23,Single(1),37
6.0627122049999995,False,35,Single(1),39

==> test.csv <==
8.375361333379999,Unknown,22,Single(1),42
7.5618555866,Unknown,20,Single(1),39
7.62578964258,Unknown,21,Single(1),42
7.62578964258,False,16,Single(1),39
6.9996768185,True,28,Single(1),39
7.1870697412,True,24,Single(1),43
7.5618555866,Unknown,16,Single(1),40

==> train.csv <==
6.75055446244,Unknown,32,Single(1),38
8.928721611,Unknown,27,Single(1),35
8.18796841068,True,27,Single(1),39
7.43839671988,Unknown,28,Single(1),38
8.3114272774,Unknown,34,Single(1),38
7.87491199864,Unknown,31,Single(1),39
7.87491199864,False,20,Single(1),39


## Lab Summary: 
In this lab, we set up the environment, sampled the natality dataset to create train, eval, test splits, and preprocessed the data in a Pandas dataframe.

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License